# Approach 

## Before version 11 is hengk's effb3 with aux loss training with 1e-3 for 10eps and 1e-4 for 5eps

    I quickly get  (3.76 + 3.92 + 3.85 + 3.7 + 3.6)/5 avg 3.766 LB: 4.44



## After Version 11 is efficientnet v2

### I keep getting CV 3.6 - 3.7 and no improve for a whole month 

    Because at first event I notice efficientnetv2 strongly overfitting, But due to lack of experiment ,

    I keep thinking use current data augments and dropout is enough for regularization , 

    I just keeping suspect that I have made some implement detail bugs in using Timm's efficientnetv2 


1. First I try attach aux loss to different layers like block 4 or block1 , add more layer or units in aux loss

    Not improve

2. Then I try different pretraining weight like tf_efficientv2_rw_* or img21k 

    Have a impact on converge speed but no improve on map

3. Then I take a divide on efficientnetv2 papers and source code for detail 

    like drop_path_rate, input stem, and efficientnetv2 new features like progress regulizer
    
4. I also try multi output like 6 classes output or freeze layers finetune
    
    
### I keep observe efficient model is keeping overfitting and very very unstable

1. I made some post in discuss then try gradient accumulate to increase batch size to make trainning more stable but not improve on map

2. So I think I made some "Simple but basicly Mistake" becasue so many people achieve better socres with efficientnetv2 seem "not many change in hengck's code "

3. So Finanly I put my attation back to overfitting , I add some data augmentation to make overfitting later

    
     So finnaly I achive about (0.392 + 0.392 + 0.385 + 0.381 + 0.374)  LB score 0.452

# Refferences:

1. hengck's efficientnetb3a with aux loss https://www.kaggle.com/c/siim-covid19-detection/discussion/240233
2. pytorch Amp with gradiant accumulate  https://pytorch.org/docs/stable/notes/amp_examples.html#gradient-accumulation
3. lovasz loss https://github.com/bermanmaxim/LovaszSoftmax
4. Data augment for effB7  https://www.kaggle.com/southsakura/kaggle

# Furthur Improvements

1. tune data augment , now data augment seems too much make model a little damping

2. shall use group stratify k-fold to avoid leaky rather than only stratify k-fold

3. a more efficient way to use aux loss or multi output

4. Transfer learning or unsupervise pretraning

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import os
import sys

import pandas as pd
import math
import csv

from timeit import default_timer as timer

from datetime import datetime

In [ ]:
# madgrad
# https://github.com/facebookresearch/madgrad

!pip install madgrad

from madgrad import MADGRAD

In [ ]:
#!pip install wandb --upgrade

In [ ]:
#import wandb
#api_key = 'e5260514cd613164ad5c3fd66ad7e909beaf820f'
#wandb.login(key=api_key)

In [ ]:
!pip install timm

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.optim import Adam

from torch.nn.parallel.data_parallel import data_parallel

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *

import collections
from collections import defaultdict

import timm
from timm.models.efficientnet import *

import torch.cuda.amp as amp

# Unpack file

In [ ]:
_data_dir = "/kaggle/input/covid19512/"

In [ ]:
!ls {_data_dir}train/000a312787f2.png

In [ ]:
mask_example = cv2.imread(_data_dir + "mask/000a312787f2.png")

In [ ]:
plt.imshow(mask_example, cmap="bone")

In [ ]:
train_example = cv2.imread(_data_dir + "train/000a312787f2.png")

In [ ]:
plt.imshow(train_example, cmap="bone")

# Prepare matedata 

In [ ]:
study_df = pd.read_csv("/kaggle/input/siim-covid19-detection/train_study_level.csv")
image_df = pd.read_csv("/kaggle/input/siim-covid19-detection/train_image_level.csv")
meta_df = pd.read_csv(_data_dir + "meta.csv")
fold_df = pd.read_csv(_data_dir + "updated_iamge_level.csv")

In [ ]:
image_df.head()

In [ ]:
_duplicateList_path = _data_dir + 'dublicate.txt'

In [ ]:
_duplicateList = []
with open(_duplicateList_path, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        _duplicateList += row

_duplicateList[:5]

# Configuration

In [ ]:
image_size = 512
data_dir = _data_dir
study_name_to_label = {
    'Negative for Pneumonia'  :0,
    'Typical Appearance'      :1,
    'Indeterminate Appearance':2,
    'Atypical Appearance'     :3,
}
study_label_to_name = { v:k for k,v in study_name_to_label.items()}
num_study_label = len(study_name_to_label)

In [ ]:
def make_fold(mode='train-0'):
    if 'train' in mode:
        df = study_df.copy()
        df.loc[:, 'id'] = df.id.str.replace('_study', '')
        df = df.rename(columns={'id': 'study_id'})
        
        fold = fold_df.copy()
        fold.loc[:, 'id'] = fold.id.str.replace('_image', '')
        fold = fold.rename(columns={'id': 'image'})

        #---
        df = df.merge(fold, left_on='study_id', right_on='StudyInstanceUID')
        
        df = df[~df['image'].isin(_duplicateList)]

        #---
        fold = int(mode[-1])
        df_train = df[df.fold != fold].reset_index(drop=True)
        df_valid = df[df.fold == fold].reset_index(drop=True)
        return df_train, df_valid

# Dataset 

In [ ]:
class SiimDataset(Dataset):
    def __init__(self, df, augment=None):
        super().__init__()
        self.df = df
        self.augment = augment
        self.length = len(df)

    def __str__(self):
        string  = ''
        string += '\tlen = %d\n'%len(self)
        string += '\tdf  = %s\n'%str(self.df.shape)

        string += '\tlabel distribution\n'
        for i in range(num_study_label):
            n = self.df[study_label_to_name[i]].sum()
            string += '\t\t %d %26s: %5d (%0.4f)\n'%(i, study_label_to_name[i], n, n/len(self.df) )
        return string


    def __len__(self):
        return self.length

    def __getitem__(self, index):
        d = self.df.iloc[index]

        #image_file = data_dir + '/%s_640/%s/%s/%s.png' % (d.set, d.study, d.series, d.image)
        image_file = data_dir + '/train/%s.png' % (d.image)
        image = cv2.imread(image_file,cv2.IMREAD_GRAYSCALE)
        onehot = d[study_name_to_label.keys()].values

        mask_file = data_dir + '/mask/%s.png' % (d.image)
        mask = cv2.imread(mask_file,cv2.IMREAD_GRAYSCALE)
        
        if str(mask) == "None":
            mask = np.zeros((image_size, image_size))

        r = {
            'index' : index,
            'd' : d,
            'image' : image,
            'mask' : mask,
            'onehot' : onehot,
        }
        if self.augment is not None: r = self.augment(r)
        return r


def null_collate(batch):
    collate = defaultdict(list)

    for r in batch:
        for k, v in r.items():
            collate[k].append(v)

    # ---
    batch_size = len(batch)
    onehot = np.ascontiguousarray(np.stack(collate['onehot'])).astype(np.float32)
    collate['onehot'] = torch.from_numpy(onehot)

    image = np.stack(collate['image'])
    image = image.reshape(batch_size, 1, image_size,image_size).repeat(3,1)
    image = np.ascontiguousarray(image)
    image = image.astype(np.float32) / 255
    collate['image'] = torch.from_numpy(image)

    mask = np.stack(collate['mask'])
    mask = mask.reshape(batch_size, 1, image_size,image_size)
    mask = np.ascontiguousarray(mask)
    mask = mask.astype(np.float32) / 255
    collate['mask'] = torch.from_numpy(mask)

    return collate

In [ ]:
# what is ascontiguousarray
#x = np.arange(6).reshape(2,3)
"""
x = np.array([[1,1,0],[2,0,1], [1,2,3,]])
y = np.ascontiguousarray(x, dtype=np.float32)
print(x.flags['C_CONTIGUOUS'])
print(x)
print(y)
"""

In [ ]:
"""
df_train, df_valid = make_fold(mode='train-1')

dataset = SiimDataset(df_valid) #null_augment
print(dataset)


for i in range(10):
    i = np.random.choice(len(dataset))
    r = dataset[i]

    print('index ' , i)
    print(r['d'])
    print(r['onehot'])
    print('')
    plt.figure()
    plt.imshow(r['image'])
    plt.figure()
    plt.imshow(r['mask'], cmap="bone")
"""

In [ ]:
"""
loader = DataLoader(
    dataset,
    sampler = RandomSampler(dataset),
    batch_size  = 8,
    drop_last   = True,
    num_workers = 0,
    pin_memory  = True,
    collate_fn  = null_collate,
)
for t,batch in enumerate(loader):
    if t>5: break

    print(t, '-----------')
    print('index : ', batch['index'])
    print('image : ')
    print('\t', batch['image'].shape, batch['image'].is_contiguous())
    print('mask : ')
    print('\t', batch['mask'].shape, batch['mask'].is_contiguous())
    print('onehot : ')
    print('\t', batch['onehot'])
    print('\t', batch['onehot'].shape, batch['onehot'].is_contiguous())
    print('')
"""

# Model 

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        #e = efficientnet_b3a(pretrained=True, drop_rate=0.3, drop_path_rate=0.2)
        #e = efficientnetv2_rw_m(pretrained=True, drop_rate=0.5, drop_path_rate=0.2)
        #e = tf_efficientnetv2_m_in21ft1k(pretrained=True, drop_rate=0.3, drop_path_rate=0.2)
        e = tf_efficientnetv2_m(pretrained=True, drop_path_rate=0.4)
        
        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1,
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head, #384, 1536
            e.bn2,
            e.act2,
        )

        self.logit = nn.Linear(1280,num_study_label)
        self.mask = nn.Sequential(
            nn.Conv2d(176, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            #nn.Conv2d(128, 128, kernel_size=3, padding=1),
            #nn.BatchNorm2d(128),
            #nn.ReLU(inplace=True),

            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        x = 2*image-1     # ; print('input ',   x.shape)
        """
        rw_m
        torch.Size([2, 32, 256, 256])
        torch.Size([2, 32, 256, 256])
        torch.Size([2, 56, 128, 128])
        torch.Size([2, 80, 64, 64])
        torch.Size([2, 152, 32, 32])
        torch.Size([2, 192, 32, 32])
        torch.Size([2, 328, 16, 16])
        torch.Size([2, 2152, 16, 16])
        """
        x = self.b0(x) ; #print (x.shape)  # torch.Size([2, 32, 256, 256])
        x = self.b1(x) ; #print (x.shape)  # torch.Size([2, 32, 256, 256])
        x = self.b2(x) ; #print (x.shape)  # torch.Size([2, 56, 128, 128])
        x = self.b3(x) ; #print (x.shape)  # torch.Size([2, 80, 64, 64])
        x = self.b4(x) ; #print (x.shape)  # torch.Size([2, 152, 32, 32])
        x = self.b5(x) ; #print (x.shape)  # torch.Size([2, 192, 32, 32])

        #------------
        mask = self.mask(x)
        #-------------

        x = self.b6(x) ; #print (x.shape)  # torch.Size([2, 328, 16, 16])
        
        x = self.b7(x) ; #print (x.shape)  # torch.Size([2, 512, 16, 16])
        
        x = self.b8(x) ; #print (x.shape)  # torch.Size([2, 2152, 16, 16])
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        x = F.dropout(x, 0.5, training=self.training)

        logit = self.logit(x)
        return logit, mask

In [ ]:
"""
batch_size = 2
C, H, W = 3, 512, 512
#C, H, W = 3, 640, 640
image = torch.randn(batch_size, C, H, W).cuda()
mask  = torch.randn(batch_size, num_study_label, H, W).cuda()

net = Net().cuda()
logit, mask = net(image)

print(image.shape)
print(logit.shape)
print(mask.shape)
"""

# Utils

In [ ]:
class Logger(object):
    def __init__(self):
        self.terminal = sys.stdout  #stdout
        self.file = None

    def open(self, file, mode=None):
        if mode is None: mode ='w'
        self.file = open(file, mode)

    def write(self, message, is_terminal=1, is_file=1 ):
        if '\r' in message: is_file=0

        if is_terminal == 1:
            self.terminal.write(message)
            self.terminal.flush()
            #time.sleep(1)

        if is_file == 1:
            self.file.write(message)
            self.file.flush()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass

In [ ]:
def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)

    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)

    else:
        raise NotImplementedError

In [ ]:
def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
        lr +=[ param_group['lr'] ]

    assert(len(lr)==1) #we support only one param_group
    lr = lr[0]

    return lr

# Lovasz loss

In [ ]:
from torch.autograd import Variable

In [ ]:
def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    p = len(gt_sorted)
    gts = gt_sorted.sum()
    intersection = gts - gt_sorted.float().cumsum(0)
    union = gts + (1 - gt_sorted).float().cumsum(0)
    jaccard = 1. - intersection / union
    if p > 1: # cover 1-pixel case
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard

# --------------------------- BINARY LOSSES ---------------------------


def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        loss = mean(lovasz_hinge_flat(*flatten_binary_scores(log.unsqueeze(0), lab.unsqueeze(0), ignore))
                          for log, lab in zip(logits, labels))
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels.float() - 1.
    errors = (1. - logits * Variable(signs))
    errors_sorted, perm = torch.sort(errors, dim=0, descending=True)
    perm = perm.data
    gt_sorted = labels[perm]
    grad = lovasz_grad(gt_sorted)
    loss = torch.dot(F.relu(errors_sorted), Variable(grad))
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = scores.view(-1)
    labels = labels.view(-1)
    if ignore is None:
        return scores, labels
    valid = (labels != ignore)
    vscores = scores[valid]
    vlabels = labels[valid]
    return vscores, vlabels

# --------------------------- multi LOSSES ---------------------------

def lovasz_softmax(probas, labels, classes='present', per_image=False, ignore=None):
    """
    Multi-class Lovasz-Softmax loss
      probas: [B, C, H, W] Variable, class probabilities at each prediction (between 0 and 1).
              Interpreted as binary (sigmoid) output with outputs of size [B, H, W].
      labels: [B, H, W] Tensor, ground truth labels (between 0 and C - 1)
      classes: 'all' for all, 'present' for classes present in labels, or a list of classes to average.
      per_image: compute the loss per image instead of per batch
      ignore: void class labels
    """
    if per_image:
        loss = mean(lovasz_softmax_flat(*flatten_probas(prob.unsqueeze(0), lab.unsqueeze(0), ignore), classes=classes)
                          for prob, lab in zip(probas, labels))
    else:
        loss = lovasz_softmax_flat(*flatten_probas(probas, labels, ignore), classes=classes)
    return loss


def lovasz_softmax_flat(probas, labels, classes='present'):
    """
    Multi-class Lovasz-Softmax loss
      probas: [P, C] Variable, class probabilities at each prediction (between 0 and 1)
      labels: [P] Tensor, ground truth labels (between 0 and C - 1)
      classes: 'all' for all, 'present' for classes present in labels, or a list of classes to average.
    """
    if probas.numel() == 0:
        # only void pixels, the gradients should be 0
        return probas * 0.
    C = probas.size(1)
    losses = []
    class_to_sum = list(range(C)) if classes in ['all', 'present'] else classes
    for c in class_to_sum:
        fg = (labels == c).float() # foreground for class c
        if (classes is 'present' and fg.sum() == 0):
            continue
        if C == 1:
            if len(classes) > 1:
                raise ValueError('Sigmoid output possible only with 1 class')
            class_pred = probas[:, 0]
        else:
            class_pred = probas[:, c]
        errors = (Variable(fg) - class_pred).abs()
        errors_sorted, perm = torch.sort(errors, 0, descending=True)
        perm = perm.data
        fg_sorted = fg[perm]
        losses.append(torch.dot(errors_sorted, Variable(lovasz_grad(fg_sorted))))
    return mean(losses)


def flatten_probas(probas, labels, ignore=None):
    """
    Flattens predictions in the batch
    """
    if probas.dim() == 3:
        # assumes output of a sigmoid layer
        B, H, W = probas.size()
        probas = probas.view(B, 1, H, W)
    B, C, H, W = probas.size()
    probas = probas.permute(0, 2, 3, 1).contiguous().view(-1, C)  # B * H * W, C = P, C
    labels = labels.view(-1)
    if ignore is None:
        return probas, labels
    valid = (labels != ignore)
    vprobas = probas[valid.nonzero().squeeze()]
    vlabels = labels[valid]
    return vprobas, vlabels

def isnan(x):
    return x != x
    
    
def mean(l, ignore_nan=False, empty=0):
    """
    nanmean compatible with generators.
    """
    l = iter(l)
    if ignore_nan:
        l = ifilterfalse(isnan, l)
    try:
        n = 1
        acc = next(l)
    except StopIteration:
        if empty == 'raise':
            raise ValueError('Empty mean')
        return empty
    for n, v in enumerate(l, 2):
        acc += v
    if n == 1:
        return acc
    return acc / n

# Training

In [ ]:
class AmpNet(Net):
    @torch.cuda.amp.autocast()
    def forward(self,*args):
        return super(AmpNet, self).forward(*args)

is_mixed_precision = True  #True #False

## Metric 

In [ ]:
from sklearn.metrics import average_precision_score

In [ ]:
def np_loss_cross_entropy(probability, truth):
    batch_size = len(probability)
    truth = truth.reshape(-1)
    p = probability[np.arange(batch_size),truth]
    loss = -np.log(np.clip(p,1e-6,1))
    loss = loss.mean()
    return loss


def np_metric_map_curve_by_class(probability, truth):
    num_sample, num_label = probability.shape
    score = []
    for i in range(num_label):
        s = average_precision_score(truth==i, probability[:,i])
        score.append(s)
    score = np.array(score)
    return score

## Augment

In [ ]:
# #--- flip ---
def do_random_hflip(image, mask):
    if np.random.rand()>0.5:
        image = cv2.flip(image,1)
        mask = cv2.flip(mask,1)
    return image, mask


# #--- geometric ---
def do_random_rotate(image, mask, mag=15 ):
    angle = np.random.uniform(-1, 1)*mag

    height, width = image.shape[:2]
    cx, cy = width // 2, height // 2

    transform = cv2.getRotationMatrix2D((cx, cy), -angle, 1.0)
    image = cv2.warpAffine(image, transform, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    mask = cv2.warpAffine(mask, transform, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)

    return image, mask


def do_random_scale( image, mask, mag=0.1 ):
    s = 1 + np.random.uniform(-1, 1)*mag
    height, width = image.shape[:2]
    w,h = int(s*width), int(s*height)
    if (h,w)==image.shape[:2]:
        return image, mask

    dst = np.array([
        [0,0],[width,height], [width,0], #[0,height],
    ]).astype(np.float32)

    if s>1:
        dx = np.random.choice(w-width)
        dy = np.random.choice(h-height)
        src = np.array([
            [-dx,-dy],[-dx+w,-dy+h], [-dx+w,-dy],#[-dx,-dy+h],#
        ]).astype(np.float32)
    if s<1:
        dx = np.random.choice(width-w)
        dy = np.random.choice(height-h)
        src = np.array([
            [dx,dy], [dx+w,dy+h], [dx+w,dy],#
        ]).astype(np.float32)

    transform = cv2.getAffineTransform(src, dst)
    image = cv2.warpAffine( image, transform, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    mask = cv2.warpAffine( mask, transform, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    return image, mask


def do_random_stretch_y( image, mask, mag=0.25 ):
    s = 1 + np.random.uniform(-1, 1)*mag
    height, width = image.shape[:2]
    h = int(s*height)
    w = width
    if h==height:
        return image, mask

    dst = np.array([
        [0,0],[width,height], [width,0], #[0,height],
    ]).astype(np.float32)


    if s>1:
        dx = 0#np.random.choice(w-width)
        dy = np.random.choice(h-height)
        src = np.array([
            [-dx,-dy],[-dx+w,-dy+h], [-dx+w,-dy],#[-dx,-dy+h],#
        ]).astype(np.float32)
    if s<1:
        dx = 0#np.random.choice(width-w)
        dy = np.random.choice(height-h)
        src = np.array([
            [dx,dy], [dx+w,dy+h], [dx+w,dy],#
        ]).astype(np.float32)

    transform = cv2.getAffineTransform(src, dst)
    image = cv2.warpAffine( image, transform, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    mask = cv2.warpAffine( mask, transform, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    return image, mask



def do_random_stretch_x( image, mask, mag=0.25 ):
    s = 1 + np.random.uniform(-1, 1)*mag
    height, width = image.shape[:2]
    h = height
    w = int(s*width)
    if w==width:
        return image, mask

    dst = np.array([
        [0,0],[width,height], [width,0], #[0,height],
    ]).astype(np.float32)

    if s>1:
        dx = np.random.choice(w-width)
        dy = 0#np.random.choice(h-height)
        src = np.array([
            [-dx,-dy],[-dx+w,-dy+h], [-dx+w,-dy],#[-dx,-dy+h],#
        ]).astype(np.float32)
    if s<1:
        dx = np.random.choice(width-w)
        dy = 0#np.random.choice(height-h)
        src = np.array([
            [dx,dy], [dx+w,dy+h], [dx+w,dy],#
        ]).astype(np.float32)

    transform = cv2.getAffineTransform(src, dst)
    image = cv2.warpAffine( image, transform, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    mask = cv2.warpAffine( mask, transform, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    return image, mask


def do_random_shift( image, mask, mag=32 ):
    b = mag
    height, width = image.shape[:2]

    image = cv2.copyMakeBorder(image, b,b,b,b, borderType=cv2.BORDER_CONSTANT, value=0)
    mask  = cv2.copyMakeBorder(mask, b,b,b,b, borderType=cv2.BORDER_CONSTANT, value=0)
    x = np.random.randint(0,2*b)
    y = np.random.randint(0,2*b)
    image = image[y:y+height,x:x+width]
    mask = mask[y:y+height,x:x+width]

    return image, mask

###########################################################################################3



# #--- noise ---
def do_random_blurout(image, size=0.20, num_cut=3):
    height, width = image.shape[:2]
    size = int(size*(height+width)/2)
    for t in range(num_cut):
        x = np.random.randint(0,width- size)
        y = np.random.randint(0,height-size)
        x0 = x
        x1 = x+size
        y0 = y
        y1 = y+size
        image[y0:y1,x0:x1]=image[y0:y1,x0:x1].mean()

    return image

def do_random_guassian_blur(image, mag=[0.1, 2.0]):
    sigma = np.random.uniform(mag[0],mag[1])
    image = cv2.GaussianBlur(image, (23, 23), sigma)
    return image

def do_random_noise(image, mag=0.08):
    height, width = image.shape[:2]

    image = image.astype(np.float32)/255
    noise = np.random.uniform(-1,1,size=(height,width))*mag
    image = image+noise

    image = np.clip(image,0,1)
    image = (image*255).astype(np.uint8)
    return image



# # --- intensity ---
def do_random_intensity_shift_contast(image, mag=[0.3,0.2]):
    image = (image).astype(np.float32)/255
    alpha0 = 1 + np.random.uniform(-1,1)*mag[0]
    alpha1 = np.random.uniform(-1,1)*mag[1]
    image = (image+alpha1)
    image = np.clip(image,0,1)
    image = image**alpha0
    image = np.clip(image,0,1)
    image = (image*255).astype(np.uint8)
    return image

#https://answers.opencv.org/question/12024/use-of-clahe/)
def do_random_clahe(image, mag=[[2,4],[6,12]]):
    l = np.random.uniform(*mag[0])
    g = np.random.randint(*mag[1])
    clahe = cv2.createCLAHE(clipLimit=l, tileGridSize=(g, g))

    image = clahe.apply(image)
    return image

# https://github.com/facebookresearch/CovidPrognosis/blob/master/covidprognosis/data/transforms.py
def do_histogram_norm(image, mag=[[2,4],[6,12]]):
    num_bin = 255

    histogram, bin = np.histogram( image.flatten(), num_bin, density=True)
    cdf = histogram.cumsum()  # cumulative distribution function
    cdf = 255 * cdf / cdf[-1]  # normalize

    # use linear interpolation of cdf to find new pixel values
    equalized = np.interp(image.flatten(), bin[:-1], cdf)
    image = equalized.reshape(image.shape)
    return image

In [ ]:
def do_valid(net, valid_loader):

    valid_probability = []
    valid_truth = []
    valid_num = 0

    net.eval()
    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        batch_size = len(batch['index'])
        image = batch['image'].cuda()
        onehot = batch['onehot']
        label = onehot.argmax(-1)

        with torch.no_grad():
            #with amp.autocast():
                logit, mask = data_parallel(net,image)
                probability = F.softmax(logit,-1)

        valid_num += batch_size
        valid_probability.append(probability.data.cpu().numpy())
        valid_truth.append(label.data.cpu().numpy())
        print('\r %8d / %d  %s'%(valid_num, len(valid_loader.dataset),time_to_str(timer() - start_timer,'sec')),end='',flush=True)

    assert(valid_num == len(valid_loader.dataset))
    #print('')
    #----------------------
    truth = np.concatenate(valid_truth)
    probability = np.concatenate(valid_probability)
    predict = probability.argsort(-1)[::-1]

    loss = np_loss_cross_entropy(probability,truth)
    topk = (predict==truth.reshape(-1,1))
    acc  = topk[:, 0]
    topk = topk.mean(0).cumsum()
    acc = [acc[truth==i].mean() for i in range(num_study_label)]

    #---
    map  = np_metric_map_curve_by_class(probability, truth)*(4/6)

    return [loss, map.mean(), topk[0], topk[1]]

In [ ]:
!pip install albumentations -U
import albumentations as alb

In [ ]:
size = 512
transform = alb.Compose([
        #位置
        alb.OneOf([
            alb.HorizontalFlip(p=0.5),
            alb.VerticalFlip(p=0.5),
        ], p=0.5),

        alb.RandomRotate90(p=.5),

        #真实杂音
        alb.OneOf([
            #alb.RandomShadow(p=0.1),  # 灰阶无法使用
            alb.CoarseDropout(p=0.2), 
            # alb.MaskDropout(p=0.02),
        ], p=0.15),

        #模糊
        alb.OneOf([
            alb.MedianBlur(blur_limit=(3, 7),p=0.5),
            alb.GlassBlur(sigma=0.9, max_delta=200, iterations=1, always_apply=False, mode='fast',p=0.05),
            alb.GaussianBlur(p=0.5),
        ], p=0.02),

        #形状
        alb.OneOf([
            alb.OpticalDistortion(p=0.3),
            alb.GridDistortion(p=0.3),
            alb.IAAPiecewiseAffine(p=0.1),
            alb.ElasticTransform (alpha=1, sigma=50,alpha_affine=40,
                              interpolation=1, border_mode=4, #value=None, mask_value=None, 
                              always_apply=False, approximate=False, p=0.3)
        ], p=0.3),

          #颜色
        alb.OneOf([
            # alb.HueSaturationValue(10,25,10, p= 0.4),  # 灰阶不兼容
            alb.CLAHE(clip_limit=4, p=0.1),
            alb.OneOf([
                #normal
                alb.RandomBrightnessContrast(brightness_limit=[-0.4,0.4], 
                                         contrast_limit=[-0.4,0.4], brightness_by_max=False, 
                                         always_apply=False, p=0.8),
                #lighting
                alb.RandomBrightnessContrast(brightness_limit=[0.4,0.6], 
                                         contrast_limit=[0.4,0.6], 
                                         brightness_by_max=False, 
                                         always_apply=False, p=0.05),
                alb.RandomBrightnessContrast(brightness_limit=[0.4,0.6], 
                                         contrast_limit=[-0.6,-0.4], brightness_by_max=False, 
                                         always_apply=False, p=0.05),
                #darking
                alb.RandomBrightnessContrast(brightness_limit=[-0.6,-0.4], 
                                         contrast_limit=[0.4,0.6], brightness_by_max=False, 
                                         always_apply=False, p=0.05),
                alb.RandomBrightnessContrast(brightness_limit=[-0.6,-0.4], 
                                         contrast_limit=[-0.5,-0.3], brightness_by_max=False, 
                                         always_apply=False, p=0.05),
            ],p=0.6)             
        ], p=0.4),

        #整体失真化
        alb.OneOf([
            alb.GaussNoise(p= 0.1),
            alb.RandomGamma (gamma_limit=(70, 110), #eps=None, 
                             always_apply=False, p=0.3),           
        ], p=0.1),

        #偏移
        alb.OneOf([
            alb.ShiftScaleRotate(shift_limit=[-0.1,0.1], scale_limit=[-0.15,0.15], rotate_limit=15, p=0.98, 
                         border_mode=cv2.BORDER_REFLECT),
            alb.ShiftScaleRotate(shift_limit=[0.1,0.3], scale_limit=[-0.15,0.15], rotate_limit=15, p=0.005, 
                         border_mode=0),
            alb.ShiftScaleRotate(shift_limit=[-0.3,0.1], scale_limit=[-0.15,0.15], rotate_limit=15, p=0.005, 
                         border_mode=0),
            alb.ShiftScaleRotate(shift_limit=0.1, scale_limit=[0.5,0.6], rotate_limit=30, p=0.01, 
                         border_mode=cv2.BORDER_REFLECT),
        ], p=0.9),
              
        alb.Cutout(num_holes=10, 
                    max_h_size=int(.1 * size), max_w_size=int(.1 * size), 
                    p=.25),
    ])
#random.seed(42)

In [ ]:
def train_augment(r):
    image = r['image']
    mask = r['mask']

    aug = transform(image=image, mask=mask)

    r['image'] = aug["image"]
    r['mask'] = aug["mask"]
    return r

In [ ]:
"""
def train_augment(r):
    image = r['image']
    mask = r['mask']
    # if image[:2].shape != (image_size, image_size):
    #     image = cv2.resize(image, dsize=(image_size, image_size), interpolation=cv2.INTER_AREA)

    if 1:
        for fn in np.random.choice([
            lambda image, mask : do_random_scale(image, mask, mag=0.20),
            lambda image, mask : do_random_stretch_y(image, mask, mag=0.20),
            lambda image, mask : do_random_stretch_x(image, mask, mag=0.20),
            lambda image, mask : do_random_shift(image, mask, mag=int(0.20*image_size)),
            lambda image, mask : (image, mask)
        ],1):
            image, mask = fn(image, mask)

        for fn in np.random.choice([
            lambda image, mask : do_random_rotate(image, mask, mag=15),
            lambda image, mask : do_random_hflip(image, mask),
            lambda image, mask : (image, mask)
        ],1):
            image, mask = fn(image, mask)

        # ------------------------
        for fn in np.random.choice([
            lambda image : do_random_intensity_shift_contast(image, mag=[0.5,0.5]),
            lambda image : do_random_noise(image, mag=0.05),
            lambda image : do_random_guassian_blur(image),
            lambda image : do_random_blurout(image, size=0.25, num_cut=2),
            #lambda image : do_random_clahe(image),
            #lambda image : do_histogram_norm(image),
            lambda image : image,
        ],1):
            image = fn(image)

    r['image'] = image
    r['mask'] = mask
    return r

"""

In [ ]:
config = {
    "image_size":image_size,
    "start_lr":1e-3,
    #fold = [0, 1, 2, 3, 4]
    "foldlist":[0],
    "batchsize":16,
    "num_iteration":18000,
    "iter_log":200,
    "iter_valid":200
    
}

In [ ]:
IDENTIFIER   = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
def run_train():
    
    #wandb.watch(Net, criterion, log="all", log_freq=10)
    
    #for fold in [0,1,2,3,4]:
    for fold in config["foldlist"]:
        
        #wandb.init(project="covid19-siim-classify", config=config)
        
        out_dir = "./"
        initial_checkpoint = None
        #out_dir + '/checkpoint/00007700_model.pth' 
            

        #start_lr   = 0.0001#1
        start_lr = config["start_lr"]
        batch_size = config["batchsize"] #14 #22

        #num_iteration = 12000
        num_iteration = config["num_iteration"]
        iter_log    = config["iter_log"]
        iter_valid  = config["iter_valid"]
        iter_save   = list(range(0, num_iteration+1, 200))
        
        accumulation_steps = 2 

        ## setup  ----------------------------------------
        for f in ['checkpoint', 'train', 'valid', 'backup']: os.makedirs(out_dir + '/' + f, exist_ok=True)
        # backup_project_as_zip(PROJECT_PATH, out_dir +'/backup/code.train.%s.zip'%IDENTIFIER)

        log = Logger()
        log.open(out_dir + '/log.train.txt', mode='a')
        log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
        #log.write('\t%s\n' % COMMON_STRING)
        #log.write('\t__file__ = %s\n' % __file__)
        log.write('\tout_dir  = %s\n' % out_dir)
        log.write('\n')

        ## dataset ------------------------------------
        df_train, df_valid = make_fold('train-%d'%fold)
        
        train_dataset = SiimDataset(df_train, train_augment)
        #train_dataset = SiimDataset(df_train, )
        valid_dataset = SiimDataset(df_valid, )

        train_loader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size,
            drop_last   = True,
            num_workers = 4,
            pin_memory  = True,
            worker_init_fn=lambda id: np.random.seed(torch.initial_seed() // 2 ** 32 + id),
            collate_fn  = null_collate,
        )
        valid_loader  = DataLoader(
            valid_dataset,
            sampler = SequentialSampler(valid_dataset),
            batch_size  = 16,
            drop_last   = False,
            num_workers = 4,
            pin_memory  = True,
            collate_fn  = null_collate,
        )

        log.write('train_dataset : \n%s\n'%(train_dataset))
        log.write('valid_dataset : \n%s\n'%(valid_dataset))
        log.write('\n')


        ## net ----------------------------------------
        log.write('** net setting **\n')
        if is_mixed_precision:
            scaler = amp.GradScaler()
            net = AmpNet().cuda()
        else:
            net = Net().cuda()


        if initial_checkpoint is not None:
            f = torch.load(initial_checkpoint, map_location=lambda storage, loc: storage)
            start_iteration = f['iteration']
            start_epoch = f['epoch']
            state_dict  = f['state_dict']
            net.load_state_dict(state_dict,strict=True)  #True
        else:
            start_iteration = 0
            start_epoch = 0


        log.write('net=%s\n'%(type(net)))
        log.write('\tinitial_checkpoint = %s\n' % initial_checkpoint)
        log.write('\n')

        # -----------------------------------------------
        if 0: ##freeze
            for p in net.block0.backbone.parameters(): p.requires_grad = False


        #optimizer = Lookahead(RAdam(filter(lambda p: p.requires_grad, net.parameters()),lr=start_lr), alpha=0.5, k=5)
        #optimizer = RAdam(filter(lambda p: p.requires_grad, net.parameters()),lr=start_lr)
        #optimizer = MADGRAD( filter(lambda p: p.requires_grad, net.parameters()), lr=start_lr, momentum= 0.9, weight_decay= 0, eps= 1e-06)
        #optimizer = Adam(net.parameters(), lr=start_lr)
        
        optimizer = MADGRAD( filter(lambda p: p.requires_grad, net.parameters()), lr=start_lr, momentum= 0.9, weight_decay= 0, eps= 1e-06)
        
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)
        
        # num_iteration = 8000
        # iter_log    = 100
        # iter_valid  = 100
        # iter_save   = list(range(0, num_iteration, 100))#1*1000

        log.write('optimizer\n  %s\n'%(optimizer))
        log.write('\n')


        ## start training here! ##############################################
        log.write('** start training here! **\n')
        log.write('   fold = %d\n'%(fold))
        log.write('   is_mixed_precision = %s \n'%str(is_mixed_precision))
        log.write('   batch_size = %d\n'%(batch_size))
        #log.write('   experiment = %s\n' % str(__file__.split('/')[-2:]))
        log.write('                      |----- VALID ---|---- TRAIN/BATCH --------------\n')
        log.write('rate     iter   epoch | loss    map  topk_0 topk_1  | loss0  loss1 loss2 | time          \n')
        log.write('----------------------------------------------------------------------\n')
                  #0.00000   0.00* 0.00  | 0.000  0.000  0.000  0.000  | 0.000  0.000  0.000 |  0 hr 00 min

        def message(mode='print'):
            if mode==('print'):
                asterisk = ' '
                loss = batch_loss
            if mode==('log'):
                asterisk = '*' if iteration in iter_save else ' '
                loss = train_loss

            text = \
                '%0.5f  %5.3f%s %4.2f  | '%(rate, iteration/10000, asterisk, epoch,) +\
                '%4.3f  %4.3f  %4.3f  %4.3f  | '%(*valid_loss,) +\
                '%4.3f  %4.3f  %4.3f  | '%(*loss,) +\
                '%s' % (time_to_str(timer() - start_timer,'min'))

            return text

        #----
        valid_loss = np.zeros(4,np.float32)
        train_loss = np.zeros(3,np.float32)
        batch_loss = np.zeros_like(train_loss)
        sum_train_loss = np.zeros_like(train_loss)
        sum_train = 0
        loss0 = torch.FloatTensor([0]).cuda().sum()
        loss1 = torch.FloatTensor([0]).cuda().sum()
        loss2 = torch.FloatTensor([0]).cuda().sum()


        start_timer = timer()
        iteration = start_iteration
        epoch = start_epoch
        rate = 0
        while  iteration < num_iteration:

            for t, batch in enumerate(train_loader):

                if iteration in iter_save:
                    if iteration != start_iteration:
                        """
                        torch.save({
                            'state_dict': net.state_dict(),
                            'iteration': iteration,
                            'epoch': epoch,
                        }, out_dir + '/checkpoint/%08d_model.pth' % (iteration))
                        """
                        
                        
                        #wandb.save(out_dir + '/checkpoint/%08d_model.pth' % (iteration))
                        
                        pass

                if (iteration % iter_valid == 0):
                    #if iteration!=start_iteration:
                        valid_loss = do_valid(net, valid_loader)  #
                        scheduler.step(valid_loss[0])
                        pass

                if (iteration % iter_log == 0):
                    print('\r', end='', flush=True)
                    log.write(message(mode='log') + '\n')
                    """
                    wandb.log({
                        "fold": fold,
                        "epoch": epoch, 
                        "rate": rate,
                        "iterate": iteration/10000,
                        "valid_loss": valid_loss[0], 
                        "valid_map": valid_loss[1],
                        "valid_topk_0": valid_loss[2], 
                        "valid_topk_1": valid_loss[3],
                        "train_ce_loss": loss0, 
                        "train_bce_loss": loss1
                    }, 
                        step=iteration)

                    """
                   

                # learning rate schduler ------------
                rate = get_learning_rate(optimizer)

                # one iteration update  -------------
                batch_size = len(batch['index'])
                image = batch['image'].cuda()
                truth_mask = batch['mask'].cuda()
                truth_mask = F.interpolate(truth_mask, size=(32,32), mode='bilinear', align_corners=False)
                #truth_mask = F.interpolate(truth_mask, size=(16,16), mode='bilinear', align_corners=False)
                onehot = batch['onehot'].cuda()
                label = onehot.argmax(-1)

                #----
                net.train()
                optimizer.zero_grad()

                if is_mixed_precision:
                    with amp.autocast():
                        logit, mask = data_parallel(net, image)
                        loss0 = F.cross_entropy(logit, label)
                        #loss1 = F.binary_cross_entropy_with_logits(mask, truth_mask)
                        
                        loss1 = F.binary_cross_entropy_with_logits(mask, truth_mask) + lovasz_softmax(torch.sigmoid(mask), truth_mask, classes=[1], ignore=255)
                        
                        loss2 = (loss1 + loss0) / accumulation_steps
                        
                    scaler.scale(loss2).backward()
                    #scaler.scale(loss1).backward()
                    #scaler.scale(loss0+loss1).backward()
                    #scaler.unscale_(optimizer)
                    #torch.nn.utils.clip_grad_norm_(net.parameters(), 2)
                    #scaler.step(optimizer)
                    #scaler.update()

                    if (iteration+1) % accumulation_steps == 0:             # Wait for several backward steps
                        scaler.unscale_(optimizer)
                        scaler.step(optimizer)                           # Now we can do an optimizer step
                        scaler.update()
                        optimizer.zero_grad()  

                else :
                    assert(False)
                    print('fp32')
                    logit, mask = data_parallel(net, image)
                    loss0 = F.cross_entropy(logit, label)
                    loss1 = F.binary_cross_entropy_with_logits(mask, truth_mask)

                    (loss0+loss1).backward()
                    optimizer.step()


                # print statistics  --------
                epoch += 1 / len(train_loader)
                iteration += 1

                batch_loss = np.array([loss0.item(), loss1.item(), loss2.item()])
                sum_train_loss += batch_loss
                sum_train += 1
                if iteration % 100 == 0:
                    train_loss = sum_train_loss / (sum_train + 1e-12)
                    sum_train_loss[...] = 0
                    sum_train = 0

                print('\r', end='', flush=True)
                print(message(mode='print'), end='', flush=True)


        log.write('\n')
        #wandb.finish()

In [ ]:
run_train()